# Bases de Datos Vectoriales para el Agro

## Setup

In [1]:
!pip install -qq chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 122.8 MB/s eta 

In [2]:
!pip install -q sentence-transformers

In [3]:
import torch
torch.__version__

'2.8.0+cu126'

In [4]:
torch.cuda.is_available()

True

## ¿Qué son los Embeddings?

Imaginen que queremos que una computadora entienda el significado de las palabras como lo hacemos nosotros. Los **embeddings** son la forma en que convertimos texto (como "sequía", "riego", "fertilizante") en números que la computadora puede procesar y comparar.

Cada palabra o frase se convierte en una lista de números llamada **vector**. Palabras con significados similares tendrán vectores parecidos. Por ejemplo:
- "sequía" y "falta de lluvia" tendrán vectores similares
- "fertilizante" y "nutriente" también serán parecidos

Esto nos permite encontrar información relacionada automáticamente.

## Ejercicio Práctico: Base de Conocimiento Agrícola

Vamos a crear una base de conocimiento con consultas técnicas de productores agropecuarios argentinos. Esto nos permitirá buscar información por significado, no solo por palabras exactas.

**Pasos que seguiremos:**

1. Crear una base de datos vectorial (ChromaDB)
2. Cargar consultas técnicas de productores
3. Realizar búsquedas por significado (búsqueda semántica)
4. Agregar más consultas y probar nuevas búsquedas

### 1. ¿Qué es ChromaDB?

ChromaDB es una base de datos especializada en almacenar vectores (listas de números que representan el significado de textos). Es como tener una biblioteca inteligente donde podemos encontrar información por su significado, no solo por palabras exactas.

Es una herramienta gratuita y fácil de usar, perfecta para empezar a experimentar con búsquedas inteligentes.

In [ ]:
import chromadb

# Creamos la base de datos in-memory para este ejercicio.
client = chromadb.Client()

### 2. Creando nuestra primera colección

Una **colección** es como una carpeta donde guardamos documentos relacionados. En nuestro caso, guardaremos consultas técnicas de productores agropecuarios.

ChromaDB se encarga automáticamente de:
- Convertir nuestros textos en vectores
- Organizarlos para búsquedas rápidas
- Permitirnos agregar, leer, actualizar y eliminar información

In [ ]:
client.list_collections()

In [ ]:
# Creamos nuestra primera colección para consultas agrícolas
collection = client.create_collection(
    name="ConsultasAgricolas",
)

In [ ]:
client.list_collections()

**¿Cómo funciona esto?**

Cuando agregamos texto a ChromaDB, automáticamente:
1. Convierte cada frase en una lista de números (vector)
2. Guarda estos números de forma organizada
3. Nos permite buscar frases similares por significado

Por defecto usa un modelo llamado **all-MiniLM-L6-v2** que funciona bien con inglés, pero después veremos uno mejor para español.

In [ ]:
# Consultas técnicas reales de productores argentinos
consultas_productores = [
    "Mi maíz tiene las hojas amarillas y no sé si es falta de nitrógeno o exceso de agua.",
    "¿Cuándo es el mejor momento para aplicar herbicida en soja en la zona núcleo?",
    "Los rindes de trigo están siendo muy bajos esta campaña, ¿puede ser por la sequía?",
    "Necesito saber si conviene hacer un análisis de suelo antes de la siembra."
]

# ChromaDB convierte automáticamente cada consulta en vectores
collection.add(
    documents=consultas_productores,
    metadatas=[{"region": "zona-nucleo"}, {"region": "zona-nucleo"}, {"region": "pampa-humeda"}, {"region": "general"}],
    ids=["consulta1", "consulta2", "consulta3", "consulta4"],
)

In [ ]:
# Verificamos que se cargaron correctamente las consultas
collection.count()

In [ ]:
# Vemos todas las consultas cargadas
collection.get()

In [ ]:
# También podemos usar el id para obtener un documento específico.
collection.get('doc2')

### 3. Búsqueda Semántica: Encontrar por Significado

In [ ]:
# Búsqueda inteligente: encuentra consultas similares por significado
pregunta_nueva = "Mi trigo está rindiendo poco, ¿será por falta de lluvia?"

results = collection.query(
    query_texts=[pregunta_nueva],
    n_results=3,
)

print("CONSULTA:", pregunta_nueva)
print("\nCONSULTAS SIMILARES ENCONTRADAS:")
for i, doc in enumerate(results['documents'][0]):
    print(f"{i+1}. {doc}")

results

In [ ]:
# Comparemos con una búsqueda tradicional por palabras exactas
collection.get(where_document={"$contains": "trigo"})

### 4. Agregar más consultas de productores

Simulemos que llegan más consultas técnicas y veamos cómo mejora nuestro sistema.

In [ ]:
nuevas_consultas = [
    "La soja está con hojas amarillentas, puede ser deficiencia nutricional?",
    "En qué momento del cultivo de maíz es mejor aplicar los agroquímicos?",
    "Este año mi cosecha de trigo fue muy mala, creo que fue por la falta de precipitaciones.",
    "¿Vale la pena hacer estudios de tierra antes de plantar?",
    "El maíz no está creciendo bien, las hojas se ven pálidas, ¿puede ser falta de fertilizante?"
]

In [ ]:
def preparar_nuevas_consultas(consultas, region, cantidad_actual):
    """Función para organizar las nuevas consultas antes de guardarlas"""
    metadatos = [{"region": region} for _ in consultas]
    nuevos_ids = [f"consulta{cantidad_actual + i + 1}" for i in range(len(consultas))]

    return {
        "documents": consultas,
        "metadatas": metadatos,
        "ids": nuevos_ids
    }

In [ ]:
cantidad_actual = collection.count()

# Preparamos las nuevas consultas de la región pampeana
nuevas_organizadas = preparar_nuevas_consultas(nuevas_consultas, "pampa-humeda", cantidad_actual)

# Las agregamos a nuestra base de datos
collection.add(**nuevas_organizadas)

In [ ]:
# Ahora tenemos más consultas en nuestra base de datos
collection.get()

In [ ]:
pregunta_nueva = "Mi cultivo tiene problemas de nutrición, las plantas se ven débiles"

results = collection.query(
    query_texts=[pregunta_nueva],
    n_results=3
)

print("NUEVA CONSULTA:", pregunta_nueva)
print("\nCONSULTAS SIMILARES ENCONTRADAS:")
for i, doc in enumerate(results['documents'][0]):
    print(f"{i+1}. {doc}")

print("\n--- RESULTADOS COMPLETOS ---")
results

In [ ]:
print("BÚSQUEDA POR PALABRAS EXACTAS:")
# Buscamos por palabras específicas
collection.get(where_document={"$or": [{"$contains": "nutrición"}, {"$contains": "fertilizante"}]})

**¿Qué observamos?**

La búsqueda semántica encuentra consultas relacionadas aunque usemos palabras diferentes. Por ejemplo, "plantas débiles" puede encontrar consultas sobre "hojas amarillas" porque ambas se refieren a problemas del cultivo.

Pero hay un problema: el modelo por defecto está entrenado principalmente en inglés, así que no entiende completamente las sutilezas del español técnico agrícola.

## Mejorando el Sistema: Modelo Multilenguaje

Para trabajar mejor con español técnico, vamos a cambiar a un modelo especializado que entiende múltiples idiomas, incluido el español agrícola argentino.

### 5. Modelo de Embeddings Multilenguaje

El modelo `multilingual-e5-large` entiende 94 idiomas diferentes, incluido el español y sus variantes técnicas. Esto significa que podrá entender mejor términos como "sequía", "rinde", "siembra", etc.

In [ ]:
from chromadb.utils import embedding_functions

# Cargamos un modelo que entiende mejor el español técnico
modelo_multilenguaje = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"
)

### 6. Nueva colección con mejor comprensión del español

In [ ]:
# client.delete_collection(name="comments_multilingual")

In [ ]:
collection_mejorada = client.get_or_create_collection(
    "ConsultasAgricolasMultilenguaje",
    embedding_function=modelo_multilenguaje,
    metadata={"hnsw:space": "cosine"}  # Mejor métrica para comparar significados
)

In [ ]:
client.list_collections()

In [ ]:
# Cargamos todas las consultas en la nueva colección mejorada
collection_mejorada.add(
    documents=consultas_productores,
    metadatas=[{"region": "zona-nucleo"}, {"region": "zona-nucleo"}, {"region": "pampa-humeda"}, {"region": "general"}],
    ids=["consulta1", "consulta2", "consulta3", "consulta4"],
)

collection_mejorada.add(
    documents=nuevas_consultas,
    metadatas=[{"region": "pampa-humeda"}, {"region": "pampa-humeda"}, {"region": "pampa-humeda"}, {"region": "pampa-humeda"}, {"region": "pampa-humeda"}],
    ids=["consulta5", "consulta6", "consulta7", "consulta8", "consulta9"],
)

### 7. Probando el sistema mejorado

In [ ]:
pregunta_test = "Mi cultivo no está creciendo bien, puede ser falta de fertilización?"

results = collection_mejorada.query(
    query_texts=[pregunta_test],
    n_results=3
)

print("CONSULTA DE PRUEBA:", pregunta_test)
print("\nCONSULTAS MÁS SIMILARES:")
for i, doc in enumerate(results['documents'][0]):
    print(f"{i+1}. {doc}")

results

**Excelente!** Ahora el sistema entiende mejor las relaciones entre conceptos en español. Puede conectar "no está creciendo bien" con "hojas amarillas", "deficiencia nutricional" y "falta de fertilizante" porque comprende que todos se refieren a problemas similares del cultivo.

## El Siguiente Paso: RAG (Retrieval Augmented Generation)

**¿Qué aprendimos hoy?**
- Los embeddings convierten texto en números que las computadoras pueden comparar
- Las bases de datos vectoriales nos permiten buscar por significado, no solo por palabras exactas
- Podemos encontrar consultas similares automáticamente, aunque usen palabras diferentes

**¿Qué viene después?**

Lo que hicimos hoy es la **primera mitad de RAG**. RAG significa "Generación Aumentada por Recuperación" y funciona así:

1. **RECUPERAR** (lo que ya hicimos): Buscar información relevante en nuestra base de datos vectorial
2. **GENERAR** (próxima clase): Usar esa información para que un LLM genere respuestas personalizadas

### Ejemplo de RAG completo:

**Consulta del productor:** "Mi soja tiene hojas amarillas, ¿qué puede ser?"

**Paso 1 - Recuperar (lo que ya sabemos hacer):**
- Buscar consultas similares en nuestra base de datos
- Encontrar: "La soja está con hojas amarillentas, puede ser deficiencia nutricional?"

**Paso 2 - Generar (próxima clase):**
- Tomar esa información relevante
- Pasársela a GPT/Claude/Gemini junto con la consulta original
- El LLM genera una respuesta técnica personalizada usando el contexto encontrado

### ¿Por qué es importante RAG?
- **Información actualizada**: Los LLMs solo conocen hasta su fecha de entrenamiento, pero RAG puede usar datos nuevos
- **Información específica**: Podemos usar nuestra propia base de conocimiento técnico
- **Respuestas precisas**: En lugar de respuestas genéricas, obtenemos respuestas basadas en casos reales similares

**La semana próxima:** Conectaremos estas búsquedas vectoriales con las APIs de OpenAI/Claude que ya conocen para crear un sistema completo de asistencia técnica agrícola.

In [ ]:
# Para la próxima clase: ejemplo simple de cómo será RAG
def ejemplo_rag_simple(consulta_productor):
    """Esta función muestra cómo funcionará RAG completo la próxima semana"""

    print("=== SIMULACIÓN RAG ===")
    print(f"Consulta del productor: {consulta_productor}")

    # PASO 1: RECUPERAR (lo que ya sabemos hacer)
    print("\n1. RECUPERANDO información similar...")
    results = collection_mejorada.query(
        query_texts=[consulta_productor],
        n_results=2
    )

    contexto = results['documents'][0]
    print(f"Consultas similares encontradas: {len(contexto)}")
    for i, doc in enumerate(contexto):
        print(f"   - {doc}")

    # PASO 2: GENERAR (próxima semana aprenderemos esto)
    print("\n2. GENERANDO respuesta personalizada...")
    print("   (Próxima clase: usaremos OpenAI/Claude con este contexto)")
    print("   Respuesta automática basada en casos similares")

    return contexto

# Probemos el concepto
ejemplo_rag_simple("¿Cuándo debo aplicar fertilizante a mi cultivo de maíz?")